In [19]:
#-*- coding:utf-8 -*-
import tensorflow as tf
#tf.enable_eager_execution()
import cv2
import os
import math
import random

import numpy as np

slim = tf.contrib.slim

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [21]:
import sys
sys.path.append('../')
print(sys.path)

['/home/zhangkailin/anaconda3/envs/tfgpu12/lib/python36.zip', '/home/zhangkailin/anaconda3/envs/tfgpu12/lib/python3.6', '/home/zhangkailin/anaconda3/envs/tfgpu12/lib/python3.6/lib-dynload', '', '/home/zhangkailin/anaconda3/envs/tfgpu12/lib/python3.6/site-packages', '/home/zhangkailin/anaconda3/envs/tfgpu12/lib/python3.6/site-packages/IPython/extensions', '/home/zhangkailin/.ipython', '../', '../', '../', '../']


In [22]:
from nets import ssd_vgg_300, ssd_common, np_methods
from preprocessing import ssd_vgg_preprocessing
from notebooks import visualization

In [24]:
# TensorFlow session: grow memory when needed. TF, DO NOT USE ALL MY GPU MEMORY!!!
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options)
isess = tf.InteractiveSession(config=config)
print(gpu_options)

allow_growth: true



## SSD 300 Model

The SSD 300 network takes 300x300 image inputs. In order to feed any image, the latter is resize to this input shape (i.e.`Resize.WARP_RESIZE`). Note that even though it may change the ratio width / height, the SSD model performs well on resized images (and it is the default behaviour in the original Caffe implementation).

SSD anchors correspond to the default bounding boxes encoded in the network. The SSD net output provides offset on the coordinates and dimensions of these anchors.

In [25]:
# Input placeholder.
net_shape = (300, 300)
data_format = 'NHWC'
img_input = tf.placeholder(tf.uint8, shape=(None, None, 3))

In [30]:
image_pre, labels_pre, bboxes_pre, bbox_img = ssd_vgg_preprocessing.preprocess_for_eval(
    img_input, None, None, net_shape, data_format, resize=ssd_vgg_preprocessing.Resize.WARP_RESIZE)
print(image_pre, '\n',labels_pre,'\n', bboxes_pre,'\n', bbox_img)
image_4d = tf.expand_dims(image_pre, 0)
print(image_4d)

Tensor("ssd_preprocessing_train_5/resize_image/Reshape:0", shape=(300, 300, 3), dtype=float32) 
 None 
 Tensor("ssd_preprocessing_train_5/strided_slice_1:0", shape=(0, 4), dtype=float32) 
 Tensor("ssd_preprocessing_train_5/strided_slice:0", shape=(4,), dtype=float32)
Tensor("ExpandDims_1:0", shape=(1, 300, 300, 3), dtype=float32)


In [32]:
# Define the SSD model.
reuse = True if 'ssd_net' in locals() else None
ssd_net = ssd_vgg_300.SSDNet()
with slim.arg_scope(ssd_net.arg_scope(data_format=data_format)):
    predictions, localisations, _, _ = ssd_net.net(image_4d, is_training=False, reuse=reuse)
print(predictions,'\n\n',localisations)

[<tf.Tensor 'ssd_300_vgg_2/softmax/Reshape_1:0' shape=(1, 38, 38, 4, 21) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/softmax_1/Reshape_1:0' shape=(1, 19, 19, 6, 21) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/softmax_2/Reshape_1:0' shape=(1, 10, 10, 6, 21) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/softmax_3/Reshape_1:0' shape=(1, 5, 5, 6, 21) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/softmax_4/Reshape_1:0' shape=(1, 3, 3, 4, 21) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/softmax_5/Reshape_1:0' shape=(1, 1, 1, 4, 21) dtype=float32>] 

 [<tf.Tensor 'ssd_300_vgg_2/block4_box/Reshape:0' shape=(1, 38, 38, 4, 4) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/block7_box/Reshape:0' shape=(1, 19, 19, 6, 4) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/block8_box/Reshape:0' shape=(1, 10, 10, 6, 4) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/block9_box/Reshape:0' shape=(1, 5, 5, 6, 4) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/block10_box/Reshape:0' shape=(1, 3, 3, 4, 4) dtype=float32>, <tf.Tensor 'ssd_300_vgg_2/

In [33]:
# Restore SSD model.
ckpt_filename = '../checkpoints/ssd_300_vgg.ckpt'
# ckpt_filename = '../checkpoints/VGG_VOC0712_SSD_300x300_ft_iter_120000.ckpt'
isess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(isess, ckpt_filename)

# SSD default anchor boxes.
ssd_anchors = ssd_net.anchors(net_shape)
print(ssd_anchors)

INFO:tensorflow:Restoring parameters from ../checkpoints/ssd_300_vgg.ckpt
[(array([[[0.01333333],
        [0.01333333],
        [0.01333333],
        ...,
        [0.01333333],
        [0.01333333],
        [0.01333333]],

       [[0.04      ],
        [0.04      ],
        [0.04      ],
        ...,
        [0.04      ],
        [0.04      ],
        [0.04      ]],

       [[0.06666667],
        [0.06666667],
        [0.06666667],
        ...,
        [0.06666667],
        [0.06666667],
        [0.06666667]],

       ...,

       [[0.94666666],
        [0.94666666],
        [0.94666666],
        ...,
        [0.94666666],
        [0.94666666],
        [0.94666666]],

       [[0.97333336],
        [0.97333336],
        [0.97333336],
        ...,
        [0.97333336],
        [0.97333336],
        [0.97333336]],

       [[1.        ],
        [1.        ],
        [1.        ],
        ...,
        [1.        ],
        [1.        ],
        [1.        ]]], dtype=float32), array([[[0.01

In [ ]:
# Input placeholder.
net_shape = (300, 300)
data_format = 'NHWC'
img_input = tf.placeholder(tf.uint8, shape=(None, None, 3))
# Evaluation pre-processing: resize to SSD net shape.
image_pre, labels_pre, bboxes_pre, bbox_img = ssd_vgg_preprocessing.preprocess_for_eval(
    img_input, None, None, net_shape, data_format, resize=ssd_vgg_preprocessing.Resize.WARP_RESIZE)
image_4d = tf.expand_dims(image_pre, 0)

# Define the SSD model.
reuse = True if 'ssd_net' in locals() else None
ssd_net = ssd_vgg_300.SSDNet()
with slim.arg_scope(ssd_net.arg_scope(data_format=data_format)):
    predictions, localisations, _, _ = ssd_net.net(image_4d, is_training=False, reuse=reuse)

# Restore SSD model.
ckpt_filename = '../checkpoints/ssd_300_vgg.ckpt'
# ckpt_filename = '../checkpoints/VGG_VOC0712_SSD_300x300_ft_iter_120000.ckpt'
isess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(isess, ckpt_filename)

# SSD default anchor boxes.
ssd_anchors = ssd_net.anchors(net_shape)

## Post-processing pipeline

The SSD outputs need to be post-processed to provide proper detections. Namely, we follow these common steps:

* Select boxes above a classification threshold;
* Clip boxes to the image shape;
* Apply the Non-Maximum-Selection algorithm: fuse together boxes whose Jaccard score > threshold;
* If necessary, resize bounding boxes to original image shape.

In [ ]:
# Main image processing routine.
def process_image(img, select_threshold=0.5, nms_threshold=.45, net_shape=(300, 300)):
    # Run SSD network.
    rimg, rpredictions, rlocalisations, rbbox_img = isess.run([image_4d, predictions, localisations, bbox_img],
                                                              feed_dict={img_input: img})
    
    # Get classes and bboxes from the net outputs.
    rclasses, rscores, rbboxes = np_methods.ssd_bboxes_select(
            rpredictions, rlocalisations, ssd_anchors,
            select_threshold=select_threshold, img_shape=net_shape, num_classes=21, decode=True)
    
    rbboxes = np_methods.bboxes_clip(rbbox_img, rbboxes)
    rclasses, rscores, rbboxes = np_methods.bboxes_sort(rclasses, rscores, rbboxes, top_k=400)
    rclasses, rscores, rbboxes = np_methods.bboxes_nms(rclasses, rscores, rbboxes, nms_threshold=nms_threshold)
    # Resize bboxes to original image shape. Note: useless for Resize.WARP!
    rbboxes = np_methods.bboxes_resize(rbbox_img, rbboxes)
    return rclasses, rscores, rbboxes

In [ ]:
# Test on some demo image and visualize output.
path = '../demo/'
image_names = sorted(os.listdir(path))

img = mpimg.imread(path + image_names[-5])
rclasses, rscores, rbboxes =  process_image(img)

# visualization.bboxes_draw_on_img(img, rclasses, rscores, rbboxes, visualization.colors_plasma)
visualization.plt_bboxes(img, rclasses, rscores, rbboxes)